# Yolo v8

Here's an example of proving Yolo v8. We will utilize the library provided by ultralytics.

In [1]:
# Install dependencies and import dependencies

# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

In [2]:
# import dependencies

import os
import json
import ezkl
import random
import numpy
from PIL import Image
from ultralytics import YOLO

We will use the smallest YOLO v8 model `yolov8n.pt` 

In [4]:
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
# Uncomment to train the model, for purpose of testing we will skip this step
# model.train(data="coco128.yaml", epochs=3)

# Evaluate model performance on a validation ste
# Uncomment to run the validation step, for purpose of testing we will skip this step
# metrics = model.val()

results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

With the model available, we can export it to onnx

In [5]:
path = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.205 🚀 Python-3.11.4 torch-2.1.0 CPU (Apple M2 Max)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 0.5s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (0.7s)
Results saved to /Users/jseam/Github/ezkl/examples/notebooks
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


In [10]:
# Specify all the files we need

model_path = os.path.join('yolov8n.onnx')
compiled_model_path = os.path.join('network.ezkl')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')
cal_data_path = os.path.join('cal_data.json')∂

In [11]:
# convert a sample input image into json

img_data = Image.open(os.path.join("bus.jpg"))
x = numpy.asarray(img_data)
data_array = x.reshape([-1]).tolist()
data = dict(input_data = [data_array])
# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

In [12]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = await ezkl.calibrate_settings(cal_data_path, model_path, settings_path, "resources")  # Optimize for resources

dimension mismatch in tensor op: conv
thread '<unnamed>' panicked at src/graph/model.rs:448:14:
called `Result::unwrap()` on an `Err` value: Synthesis
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: Synthesis

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
res = ezkl.get_srs(srs_path, settings_path)

In [ ]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )
 

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# Generate the Witness for the proof

# now generate the witness file
witness_path = os.path.join('witness.json')

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# Generate the proof

proof_path = os.path.join('proof.json')

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

print(proof)
assert os.path.isfile(proof_path)

In [ ]:
# verify our proof

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")